In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [2]:
train_processed_sample = 'data/processed/train_processed_sample.csv'
train_processed_target = 'data/processed/train_processed_target.csv'

test_processed_sample = 'data/processed/test_processed_sample.csv'
test_processed_target = 'data/processed/test_processed_target.csv'

validate_processed_sample = 'data/processed/validate_processed_sample.csv'
validate_processed_target = 'data/processed/validate_processed_target.csv'

In [3]:
X_train = pd.read_csv(train_processed_sample)
y_train = pd.read_csv(train_processed_target)

X_test = pd.read_csv(test_processed_sample)
y_test = pd.read_csv(test_processed_target)

X_val = pd.read_csv(validate_processed_sample)
y_val = pd.read_csv(validate_processed_target)

In [4]:
# Create DMatrix for model
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'multi:softprob',  # Multi-class classification with probabilities
    'num_class': y_train.shape[1],  # Number of classes
    'eval_metric': 'mlogloss',  # Multi-class log loss
    'max_depth': 6,  # Maximum depth of a tree
    'learning_rate': 0.1,  # Learning rate
    'subsample': 0.8,  # Subsample ratio
    'colsample_bytree': 0.8,  # Subsample ratio of columns
    'device': 'cuda',
}

bst = xgb.train(params, dtrain, num_boost_round=100)

y_pred_prob = bst.predict(dtest)

# Evaluate the model
logloss = log_loss(y_test, y_pred_prob)
print(f'Log Loss: {logloss:.4f}')

print('Predicted Probabilities:')
print(pd.DataFrame(y_pred_prob, columns=[f'Class_{i}' for i in range(y.shape[1])]))

# Save the model to a file
model_filename = 'model/xgboost_model_1.json'
bst.save_model(model_filename)

XGBoostError: [15:09:13] /workspace/include/xgboost/objective.h:101: multioutput is not supported by the current objective function
Stack trace:
  [bt] (0) /home/LENOVO/code/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x22dbbc) [0x7fc5e271cbbc]
  [bt] (1) /home/LENOVO/code/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x5c2982) [0x7fc5e2ab1982]
  [bt] (2) /home/LENOVO/code/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x5c2ad9) [0x7fc5e2ab1ad9]
  [bt] (3) /home/LENOVO/code/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x5c92a2) [0x7fc5e2ab82a2]
  [bt] (4) /home/LENOVO/code/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x5cabbd) [0x7fc5e2ab9bbd]
  [bt] (5) /home/LENOVO/code/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x6f) [0x7fc5e262642f]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.7(+0x6ff5) [0x7fc62f1c2ff5]
  [bt] (7) /lib/x86_64-linux-gnu/libffi.so.7(+0x640a) [0x7fc62f1c240a]